<a href="https://colab.research.google.com/github/aravind-gk/Deep-Learning-for-NLP-Assignments/blob/main/Assignment-5/DAN_BBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [ ]:
import os
import numpy as np
import contractions as ctc
import re

In [ ]:
from tensorflow.keras import models,layers
import tensorflow.keras.backend as K
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model

In [ ]:
from sklearn.metrics import classification_report,f1_score

# **Load data**


In [ ]:
df_train=pd.read_csv("./TrainData.csv")

In [ ]:
labels={'business':0, 'tech':1, 'politics':2, 'sport':3, 'entertainment':4}

In [ ]:
df_train["labels"]=df_train["Category"].map(labels)

In [ ]:
df_train.head()

,Text,Category,labels
0,worldcom ex-boss launches defence lawyers defe...,business,0
1,german business confidence slides german busin...,business,0
2,bbc poll indicates economic gloom citizens in ...,business,0
3,lifestyle governs mobile choice faster bett...,tech,1
4,enron bosses in $168m payout eighteen former e...,business,0


In [ ]:
df_train.Text[0]

'worldcom ex-boss launches defence lawyers defending former worldcom chief bernie ebbers against a battery of fraud charges have called a company whistleblower as their first witness.  cynthia cooper  worldcom s ex-head of internal accounting  alerted directors to irregular accounting practices at the us telecoms giant in 2002. her warnings led to the collapse of the firm following the discovery of an $11bn (£5.7bn) accounting fraud. mr ebbers has pleaded not guilty to charges of fraud and conspiracy.  prosecution lawyers have argued that mr ebbers orchestrated a series of accounting tricks at worldcom  ordering employees to hide expenses and inflate revenues to meet wall street earnings estimates. but ms cooper  who now runs her own consulting business  told a jury in new york on wednesday that external auditors arthur andersen had approved worldcom s accounting in early 2001 and 2002. she said andersen had given a  green light  to the procedures and practices used by worldcom. mr ebb

In [ ]:
def is_special(text):
    rem = ''
    for i in text: 
        if i.isalnum():
            rem = rem + i
        else:
            rem = rem + ' '
            rem=rem+i
            rem = rem + ' '
    return rem

In [ ]:
def rem_extra(text):
    rem=re.sub(' +', ' ',text)
    return str(rem).strip()

In [ ]:
df_train.Text=df_train.Text.apply(is_special)

In [ ]:
df_train.Text=df_train.Text.apply(rem_extra)

# **Glove embedding**

In [ ]:
embeddings_index = dict()
addrs="../Embeddings/glove.6B.300d.txt"
f = open(addrs)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [ ]:
x_train=np.zeros((len(df_train.Text),300))
k=0
for review in df_train.Text:
    vec=np.zeros((300))
    c=0
    for word in review.split(): 
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
    x_train[k]=vec/c
    k+=1
    

In [ ]:
x_train.shape

(1490, 300)

# **Basic Pre-processing**

In [ ]:
df_test=pd.read_csv("./TestData_Inputs.csv")

In [ ]:
df_test.head()

,ArticleId,Text
0,1018,qpr keeper day heads for preston queens park r...
1,1319,software watching while you work software that...
2,1138,d arcy injury adds to ireland woe gordon d arc...
3,459,india s reliance family feud heats up the ongo...
4,1020,boro suffer morrison injury blow middlesbrough...


In [ ]:
df_test.Text=df_test.Text.apply(is_special)

In [ ]:
df_test.Text=df_test.Text.apply(rem_extra)

In [ ]:
x_test=np.zeros((len(df_test.Text),300))
k=0
for review in df_test.Text:
    vec=np.zeros((300))
    c=0
    for word in review.split(): 
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            vec+=embedding_vector
            c+=1
    x_test[k]=vec/c
    k+=1
    

In [ ]:
df_labels=pd.read_csv("./TestLabels.csv")

In [ ]:
df_labels["labels"]=df_labels["Category"].map(labels)

# **DAN using Glove**


In [ ]:
model=Sequential()
model.add(InputLayer((300)))
model.add(Dense(128,activation="swish"))
model.add(Dense(128,activation="swish"))
model.add(Dense(128,activation="swish"))
model.add(Dense(5,activation="softmax"))

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics="accuracy")

In [ ]:
early=keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')

In [ ]:
model.fit(x_train, df_train.labels.values, epochs=30, batch_size=64, validation_split=0.2, callbacks=[early])

Epoch 1/100
42/42 [==============================] - 1s 12ms/step - loss: 2.6953 - accuracy: 0.5720 - val_loss: 0.7380 - val_accuracy: 0.7383
Epoch 2/100
42/42 [==============================] - 0s 3ms/step - loss: 0.6403 - accuracy: 0.8009 - val_loss: 0.7357 - val_accuracy: 0.7919
Epoch 3/100
42/42 [==============================] - 0s 3ms/step - loss: 0.4050 - accuracy: 0.8606 - val_loss: 0.3793 - val_accuracy: 0.8591
Epoch 4/100
42/42 [==============================] - 0s 3ms/step - loss: 0.2209 - accuracy: 0.9262 - val_loss: 0.2810 - val_accuracy: 0.9060
Epoch 5/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1614 - accuracy: 0.9485 - val_loss: 0.3800 - val_accuracy: 0.8658
Epoch 6/100
42/42 [==============================] - 0s 4ms/step - loss: 0.1880 - accuracy: 0.9389 - val_loss: 0.3758 - val_accuracy: 0.8859
Epoch 7/100
42/42 [==============================] - 0s 3ms/step - loss: 0.1520 - accuracy: 0.9493 - val_loss: 0.3130 - val_accuracy: 0.9262
Epoch 8/100


In [ ]:
y_pred=np.argmax(model.predict(x_test),axis=1)
print(classification_report(df_labels.labels.values,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.91      0.94       176
           1       0.92      0.98      0.95       135
           2       0.95      0.98      0.96       143
           3       0.99      0.98      0.98       168
           4       0.97      0.96      0.97       113

    accuracy                           0.96       735
   macro avg       0.96      0.96      0.96       735
weighted avg       0.96      0.96      0.96       735



In [ ]:
f1_score(df_labels.labels.values,y_pred)

0.9608977969125032